# Covid-19 ... noch mehr Diagramme
Nach einer Idee und einem Script von [Tiba Razmi](https://www.linkedin.com/pulse/covid-19-swiss-model-prediction-tiba-razmi/)  
Covid-19 Daten aus [Kaggle](https://www.kaggle.com/imdevskp/corona-virus-report)  
Covid-19 Daten für das Tessin [TI](https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_der_Schweiz) und die [Lombardei](https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Italien) aus wikipedia.org  
Bevölkerungsdaten der Länder aus [Worldometers.info](https://www.worldometers.info/world-population/population-by-country/)  
weitere Quellen: [worldometers](https://www.worldometers.info/coronavirus/)  
Bevölkerungsdaten Tessin und Lombardei aus Wikipedia.  
* *click on legend to hide curves*
* *use mouse to zoom in*

In [2]:
'''md
## Daten importieren und vorbereiten
'''
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# hide warnings
import warnings
warnings.filterwarnings('ignore')
''' md
## Daten aus Kaggle einlesen
'''
df = pd.read_csv('complete_data_new_format.csv', delimiter=',')
#nRow, nCol = df.shape
#print(f'There are {nRow} rows and {nCol} columns')
df.Date = pd.to_datetime(df.Date)
#print(df.dtypes)
#df.head()
''' md
### Bevölkerungs-Daten einlesen
'''

pop=pd.read_csv('population.csv', delimiter=';')
pop_dict = dict(zip(pop["country"],pop["population"]))

'''  md
## Weitere Kennzahlen berechnen und im Dataframe ergänzen

| Parameter | Bemerkung |
| :--- | :--- |
| df_selec | dataframe mit ausgewählten Ländern |
| mortality\_% | death / confirmed |
| extent_% | confirmed / population |
| selected_country | Einige Länder sind in Province/State unterteilt. Wenn Überseegebiete nicht zu berücksichtigen sind, z.B. Frankreich, ist <br>**df["Province/State"] == selected_country** <br>zu verwenden |


### Land wählen ('Country/Region' oder 'Province/State')
Einige Länder sind in Province/State unterteilt. Wenn Überseegebiete nicht zu berücksichtigen sind, z.B. Frankreich, ist 
**df["Province/State"]** zu verwenden

* Um Länder anzuzeigen und zu sehen, ob ein Land in "Provinzen" unterteilt ist: folgende Zeilen ausführen:
'''

#df["Country/Region"].unique()
#df[df["Country/Region"].isin(["France"])]
''' md
* Jetzt hier die Länder wählen (max 6):
'''

#                                                                                                                  select country
c_s = ["TI","Italy","Switzerland","Lombardia","South Korea","Spain", "Germany", "Austria", "France", "US"]
#states = ["France"]             # Datenstruktur hat geändert. Hier anpassen
df_sel = df[df["Province/State"].isnull()]
df_selec = df_sel[df_sel["Country/Region"].isin(c_s)]
#df_states = df[df["Province/State"].isin(states)] 
#df_selec=df_countries
#df_selec = pd.concat([df_countries, df_states], axis=0)
#c_s = countries + states


''' md
### Kennzahlen einfügen
'''

df_selec['Mortality_%'] = df_selec.Deaths / df_selec.Confirmed * 100
#df_selec.head()

#df['Country/Region']
#pop_dict[df['Country/Region'][3]]
df_selec['Pop'] = df_selec['Country/Region']
df_selec['Pop']=df_selec['Pop'].map(pop_dict)
df_selec['Extent_%'] = df_selec.Confirmed / df_selec.Pop * 100
df_selec['Deaths%pop']=df_selec.Deaths / df_selec.Pop * 100
df_selec=df_selec.sort_values(by=['Date'])
#tmp = df_selec[df_selec["Country/Region"]=="US"]
#print(tmp)
#df_selec

'''md
# Plotly
'''

# Quelle: https://www.youtube.com/watch?v=XUNaGFa9xCM
# dort gibt es ein weiteres Beispiel mit Widgets und interactive
#df.loc[df['Country/Region'] == "Italy"]

import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
py.offline.init_notebook_mode(connected=True)
#                                                                                   Fig. (a)
layouta = go.Layout(
    title='Mortality in % of confirmed cases',
    yaxis=dict(
        title='% of confirmed cases'
    ),
)

trace0a = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[0],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[0], "Mortality_%"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1a = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[1],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[1],"Mortality_%"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2a = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[2],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[2], "Mortality_%"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3a = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[3],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[3],"Mortality_%"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4a = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[4],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[4], "Mortality_%"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5a = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[5],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[5],"Mortality_%"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace6a = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[6],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[6],"Mortality_%"],
    mode='lines',
    name=c_s[6],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace7a = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[7],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[7],"Mortality_%"],
    mode='lines',
    name=c_s[7],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace8a = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[8],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[8],"Mortality_%"],
    mode='lines',
    name=c_s[8],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace9a = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[9],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[9],"Mortality_%"],
    mode='lines',
    name=c_s[9],
    line=dict(
        shape='spline',
        dash='dot'
    )
)

#                                                                                   Fig. (b)
layoutb = go.Layout(
    title='confirmed cases in % of population',
    yaxis=dict(
        title='% of population'
    ),
)

trace0b = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[0],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[0], "Extent_%"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1b = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[1],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[1],"Extent_%"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2b = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[2],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[2], "Extent_%"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3b = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[3],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[3],"Extent_%"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4b = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[4],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[4], "Extent_%"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5b = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[5],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[5],"Extent_%"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace6b = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[6],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[6],"Extent_%"],
    mode='lines',
    name=c_s[6],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace7b = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[7],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[7],"Extent_%"],
    mode='lines',
    name=c_s[7],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace8b = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[8],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[8],"Extent_%"],
    mode='lines',
    name=c_s[8],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace9b = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[9],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[9],"Extent_%"],
    mode='lines',
    name=c_s[9],
    line=dict(
        shape='spline',
        dash='dot'
    )
)

#                                                                                   Fig. (c)

layoutc = go.Layout(
    title='Deaths in % of population',
    yaxis=dict(
        title='% of population'
    ),
)

trace0c = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[0],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[0], "Deaths%pop"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1c = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[1],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[1],"Deaths%pop"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2c = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[2],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[2], "Deaths%pop"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3c = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[3],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[3],"Deaths%pop"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4c = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[4],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[4], "Deaths%pop"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5c = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[5],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[5],"Deaths%pop"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace6c = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[6],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[6],"Deaths%pop"],
    mode='lines',
    name=c_s[6],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace7c = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[7],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[7],"Deaths%pop"],
    mode='lines',
    name=c_s[7],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace8c = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[8],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[8],"Deaths%pop"],
    mode='lines',
    name=c_s[8],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace9c = go.Scatter(
    x=df_selec.loc[df['Country/Region'] == c_s[9],"Date"],
    y=df_selec.loc[df['Country/Region'] == c_s[9],"Deaths%pop"],
    mode='lines',
    name=c_s[9],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
figb = go.Figure(data=[trace0b, trace1b, trace2b, trace3b, trace4b, trace5b, trace6b, trace7b, trace8b, trace9b], layout=layoutb)
py.offline.iplot(figb)

figa = go.Figure(data=[trace0a, trace1a, trace2a, trace3a, trace4a, trace5a, trace6a, trace7a, trace8a, trace9a], layout=layouta)
py.offline.iplot(figa)

figc = go.Figure(data=[trace0c, trace1c, trace2c, trace3c, trace4c, trace5c, trace6c, trace7c, trace8c, trace9c], layout=layoutc)
py.offline.iplot(figc)
